In [3]:
import os
import json
import pickle
import numpy as np
import pandas as pd
import tweepy as tw
from tqdm import tqdm

In [4]:
def create_batch(data, batch_size=32):
    for idx in range(0, len(data), batch_size):
        yield data[idx:idx+batch_size].tolist()

In [ ]:
crowdbreaks_data = pd.read_csv("data/crowdbreaks_data/crowdbreaks_data.csv")

with open('src/twitter_credentials.json', 'rb') as f:
    credentials = json.load(f)

In [38]:
auth = tw.OAuthHandler(credentials['consumer_key'], credentials['consumer_secret'])
api = tw.API(auth)

In [27]:
batches = create_batch(crowdbreaks_data['tweet_id'].values, 100)
for counter, batch in tqdm(enumerate(list(batches))):
    data_saving_path = f"data/crowdbreaks_data/batch_{counter}.db"
    if os.path.isfile(data_saving_path):
        continue
    call_ = api.lookup_statuses(batch)
    
    with open(data_saving_path, 'wb') as f:
        pickle.dump(call_, f)

280it [04:29,  1.04it/s]


## Db to csv

In [9]:
db_path = "data/crowdbreaks_data/raw_db"
db_list = os.listdir(db_path)

In [58]:
text = []
for file_name in db_list:
    with open(f"{db_path}/{file_name}", 'rb') as f:
        db_obj = pickle.load(f)
    for status in db_obj:
        text.append([status.id_str, status.text])

In [60]:
ids_and_labels = pd.read_csv("data/crowdbreaks_data/crowdbreaks_tweets_ids_and_labels.csv", dtype=str)

In [61]:
ids_and_labels_hashmap = {}
for i in ids_and_labels.values:
    ids_and_labels_hashmap[i[0]] = i[-1]

In [62]:
for idx in range(len(text)):
    text[idx].append(ids_and_labels_hashmap[text[idx][0]])

In [64]:
pd.DataFrame(np.array(text), columns=['id', 'text', 'label']).to_csv(
    "data/crowdbreaks_data/crowdbreaks_tweets.csv", index=False)